In [ ]:
!pip install llama-index -q -q

In [ ]:
!pip install tavily-python -q -q

In [ ]:
from tavily import AsyncTavilyClient
from llama_index.core.agent.workflow import AgentWorkflow
from llama_index.llms.openai import OpenAI
import os
from openai import OpenAI as OpenAIClient

raw_client = OpenAIClient()

API_KEY   = raw_client.api_key
API_BASE  = raw_client.base_url

tavily_api_key = os.environ["TAVILY_API_KEY"]

async def search_web(query: str) -> str:
    """Useful for using the web to answer questions."""
    client = AsyncTavilyClient(api_key=tavily_api_key)
    return str(await client.search(query))

llm = OpenAI(model="gpt-4o-mini", api_key=API_KEY, api_base=API_BASE)

workflow = AgentWorkflow.from_tools_or_functions(
    [search_web],
    llm=llm,
    system_prompt="You are a helpful assistant that answers questions. If you don't know the answer, you can search the web for information.",
)

In [ ]:
from llama_index.core.agent.workflow import (
    AgentInput,
    AgentOutput,
    ToolCall,
    ToolCallResult,
    AgentStream,
)

handler = workflow.run(user_msg="What is the weather in Saskatoon?")

async for event in handler.stream_events():
    if isinstance(event, AgentStream):
        print(event.delta, end="", flush=True)

The current weather in Saskatoon is as follows:

- **Temperature**: 26.3°C (79.3°F)
- **Condition**: Sunny
- **Feels Like**: 27.1°C (80.7°F)
- **Wind**: 7.6 mph (12.2 kph) from the WSW
- **Humidity**: 48%
- **Visibility**: 19 km (11 miles)
- **Pressure**: 1017.0 mb (30.03 in)

Overall, it's a sunny day with a slight chance of showers later in the afternoon. 

For more details, you can check the full weather report [here](https://www.weatherapi.com/).

turning on more logging and running it again.

In [ ]:
from llama_index.core.agent.workflow import (
    AgentInput,
    AgentOutput,
    ToolCall,
    ToolCallResult,
    AgentStream,
)

handler = workflow.run(user_msg="What is the weather in Saskatoon?")

async for event in handler.stream_events():
    if isinstance(event, AgentInput):
       print("Agent input: ", event.input)  # Current input messages
       print("Agent name:", event.current_agent_name)  # Current agent name
    elif isinstance(event, AgentOutput):
       print("Agent output: ", event.response)  # Current full response
       print("Tool calls made: ", event.tool_calls)  # Selected tool calls, if any
       print("Raw LLM response: ", event.raw)  # Raw llm api response
    elif isinstance(event, ToolCallResult):
       print("Tool called: ", event.tool_name)  # Tool name
       print("Arguments to the tool: ", event.tool_kwargs)  # Tool kwargs
       print("Tool output: ", event.tool_output)  # Tool output

Agent input:  [ChatMessage(role=<MessageRole.SYSTEM: 'system'>, additional_kwargs={}, blocks=[TextBlock(block_type='text', text="You are a helpful assistant that answers questions. If you don't know the answer, you can search the web for information.")]), ChatMessage(role=<MessageRole.USER: 'user'>, additional_kwargs={}, blocks=[TextBlock(block_type='text', text='What is the weather in Saskatoon?')])]
Agent name: Agent
Agent output:  assistant: 
Tool calls made:  [ToolSelection(tool_id='call_a0qqgpnO0SdnH5LoNBrAqqX9', tool_name='search_web', tool_kwargs={'query': 'current weather in Saskatoon'})]
Raw LLM response:  {'id': 'chatcmpl-CAhfD5EhfRxj0h9F3PPlK5JBHRR6N', 'choices': [{'delta': {'content': None, 'function_call': None, 'refusal': None, 'role': None, 'tool_calls': None}, 'finish_reason': 'tool_calls', 'index': 0, 'logprobs': None}], 'created': 1756667159, 'model': 'gpt-4o-mini-2024-07-18', 'object': 'chat.completion.chunk', 'service_tier': 'default', 'system_fingerprint': 'fp_560a

We can see the agent accepting input, picking the web search tool, calling it with appropriate arguments, and returning the output.